In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
from gluonts.dataset.common import ListDataset
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

In [ ]:
# Parameters and settings

# Parameters for data split
WINDOW = 21    # rolling window size to use as predictors
DATE_COL = 'Date'
ID_COL = 'PERMNO'
TARGET_COL = 'excess_return'

# File path for the cleaned and filtered data file
current_directory = os.getcwd()
clean_filtered_data_path = os.path.join(current_directory, 'Data', 'clean_filtered_data.csv')

# File path to save prediction results
results_path = os.path.join(current_directory, 'Results', f'uni2ts_models_results{WINDOW:.0f}.csv')

# Estimation (in sample) period dates
in_sample_start_date = pd.to_datetime("2000-01-01")
in_sample_end_date = pd.to_datetime("2015-12-31")

# Out-of-sample period dates
out_sample_start_date = pd.to_datetime("2016-01-01")
out_sample_end_date = pd.to_datetime("2024-12-31")

# Use GPU if available, else default to using CPU
device_map = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1: Load and Process Data

In [ ]:
# Load the cleaned and filtered data files for in sample and out of sample periods into a pandas DataFrames
df = pd.read_csv(clean_filtered_data_path)

# Ensure the date columns are in datetime format
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df[[ID_COL, DATE_COL, TARGET_COL]].dropna()
df = df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)

df.info()

In [ ]:
# Check number of unique stocks
stocks_permno = df["PERMNO"].unique().tolist()
print(f"Number of unique stocks: {len(stocks_permno)}")

In [ ]:
# Create rolling window for predictors
for lag in range(1, WINDOW+1):
    df[f'lag_{lag}'] = df.groupby(ID_COL)[TARGET_COL].shift(lag)
df = df.dropna(subset=[f'lag_{lag}' for lag in range(1, WINDOW+1)]).reset_index(drop=True)

df.info()

In [ ]:
# Spit data into estimation (in-sample) and out-of-sample data
df_train = df[(df[DATE_COL] >= in_sample_start_date) & (df[DATE_COL] <= in_sample_end_date)].copy().reset_index(drop=True)
df_test = df[(df[DATE_COL] >= out_sample_start_date) & (df[DATE_COL] <= out_sample_end_date)].copy().reset_index(drop=True)

print(df_train.info())
print(df_test.info())

In [ ]:
# Create rolling window for predictors
records = []
for _, row in df_test.iterrows():
    context = [row[f'lag_{i}'] for i in range(WINDOW, 0, -1)]
    start_ts = row[DATE_COL] - pd.Timedelta(days=WINDOW)
    records.append({
        "start":  start_ts,
        "target": context
    })

test_ds = ListDataset(records, freq="D")

In [ ]:
y_test = pd.Series(df_test[TARGET_COL].values)

results = df_test[[ID_COL, DATE_COL, TARGET_COL]]

### Step 2: Zero-Shot Forecasting with Uni2TS-Moirai

In [ ]:
# Creating a Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [ ]:
# Zero Shot Uni2ts-Moirai Small
moirai_s = MoiraiForecast(
    module = MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small"),
    prediction_length = 1,
    context_length = WINDOW,
    patch_size = "auto",
    num_samples = 100,
    target_dim = 1,
    feat_dynamic_real_dim = 0,
    past_feat_dynamic_real_dim = 0
)
predictor = moirai_s.create_predictor(batch_size=32)
predictor.to(device)

preds = [forecasts.mean[0] for forecasts in predictor.predict(test_ds)]
y_moirai_s = pd.Series(preds)

results['y_moirai_s'] = y_moirai_s

### Step 3: Zero-Shot Forecasting with Uni2TS-Moirai-Moe

In [ ]:
# Zero Shot Uni2ts-Moirai-MoE Small
moirai_moe_s = MoiraiMoEForecast(
    module = MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-small"),
    prediction_length = 1,
    context_length = WINDOW,
    patch_size = 16,
    num_samples = 100,
    target_dim = 1,
    feat_dynamic_real_dim = 0,
    past_feat_dynamic_real_dim = 0
)
predictor = moirai_moe_s.create_predictor(batch_size=32)
predictor.to(device)

preds = [forecasts.mean[0] for forecasts in predictor.predict(test_ds)]
y_moirai_moe_s = pd.Series(preds)

results['y_moirai_moe_s'] = y_moirai_moe_s

In [ ]:
# Zero Shot Uni2ts-Moirai-MoE Base
moirai_moe_b = MoiraiMoEForecast(
    module = MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-base"),
    prediction_length = 1,
    context_length = WINDOW,
    patch_size = 16,
    num_samples = 100,
    target_dim = 1,
    feat_dynamic_real_dim = 0,
    past_feat_dynamic_real_dim = 0
)
predictor = moirai_moe_b.create_predictor(batch_size=32)
predictor.to(device)

preds = [forecasts.mean[0] for forecasts in predictor.predict(test_ds)]
y_moirai_moe_b = pd.Series(preds)

results['y_moirai_moe_b'] = y_moirai_moe_b

### Step 4: Evaluate Statistical Performance of Models

In [ ]:
# Evaluate models

# Unit2TS-Moirai-Small
r2_moirai_s  = r2(y_test, y_moirai_s)
mse_moirai_s = mean_squared_error(y_test, y_moirai_s)
mae_moirai_s = mean_absolute_error(y_test, y_moirai_s)
da_moirai_s = (np.sign(y_test) == np.sign(y_moirai_s)).mean()

# Unit2TS-Moirai-MoE-Small
r2_moirai_moe_s  = r2(y_test, y_moirai_moe_s)
mse_moirai_moe_s = mean_squared_error(y_test, y_moirai_moe_s)
mae_moirai_moe_s = mean_absolute_error(y_test, y_moirai_moe_s)
da_moirai_moe_s = (np.sign(y_test) == np.sign(y_moirai_moe_s)).mean()

# Unit2TS-Moirai-MoE-Base
r2_moirai_moe_b  = r2(y_test, y_moirai_moe_b)
mse_moirai_moe_b = mean_squared_error(y_test, y_moirai_moe_b)
mae_moirai_moe_b = mean_absolute_error(y_test, y_moirai_moe_b)
da_moirai_moe_b = (np.sign(y_test) == np.sign(y_moirai_moe_b)).mean()

In [ ]:
# Collating Results

results_matrix = [{
        "Model": "Uni2ts-Moirai Small",
        "R-squared": r2_moirai_s,
        "MSE": mse_moirai_s,
        "MAE": mae_moirai_s,
        "Direction Accuracy": da_moirai_s
    },
    {
        "Model": "Uni2ts-Moirai-MoE Small",
        "R-squared": r2_moirai_moe_s,
        "MSE": mse_moirai_moe_s,
        "MAE": mae_moirai_moe_s,
        "Direction Accuracy": da_moirai_moe_s
    },
    {
        "Model": "Uni2ts-Moirai-MoE Base",
        "R-squared": r2_moirai_moe_b,
        "MSE": mse_moirai_moe_b,
        "MAE": mae_moirai_moe_b,
        "Direction Accuracy": da_moirai_moe_b
    }]

results_matrix_df = pd.DataFrame(results_matrix)
results_matrix_df

##### Save Results

In [ ]:
# Save Prediction Results
results.to_csv(results_path, index=False)